## Notebook Purpose

Recreate deep decoder experiments run in `ConvDecoder_vs_DIP_vs_DD_multicoil.ipynb`, hereon referred to as the original notebook, which was extremely messy and unnecessarily complicated.

In [1]:
import os, sys
import h5py
import numpy as np
import torch
import matplotlib.pyplot as plt

from utils.transform import np_to_tt, np_to_var, apply_mask, ifft_2d
from utils.helpers import num_params
from include.decoder_conv import convdecoder
from include.mri_helpers import get_scale_factor
from include.helpers import np_to_var
from include.fit import fit

# TODO: fix these imports
# from include import * 
# from include import transforms as transform
# from common.evaluate import * 
# from torch.autograd import Variable

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    dtype = torch.cuda.FloatTensor
    torch.cuda.set_device(0)
#     print("num GPUs",torch.cuda.device_count())
else:
    dtype = torch.FloatTensor

### Load MRI measurements, y

Isolate individual 2D slice

In [3]:
filename = '/bmrNAS/people/dvv/multicoil_test_v2/file1000781_v2.h5'
f = h5py.File(filename, 'r') 
# print('h5 file keys: ', f.keys())
print('k-space shape (num_slices, num_coils, x, y): ', f['kspace'].shape)

# isolate central k-space slice
slice_idx = f['kspace'].shape[0] // 2
slice_ksp = f['kspace'][slice_idx]
# note: didn't add tensor version e.g. slice_ksp_torchtensor in original

k-space shape (num_slices, num_coils, x, y):  (37, 15, 640, 368)


### Load mask, M

- Format of loaded mask is 1d binary vector of size ~368, i.e. sampling of vertical lines in image
- Convert mask to 0's and 1's, zero pad, convert to 2D, create torch transform

Note: See original notebook for generating a new mask, e.g. if .h5 doesn't have a mask

In [4]:
try:
    mask1d = np.array([1 if e else 0 for e in f["mask"]]) # load 1D binary mask
except:
    print('Implement method for generating a mask')
    sys.exit()
    
# zero out mask in outer regions e.g. mask and data have last dimn 368, but actual data is size 320
# TODO: if actual data is size 320, then why do we have dimn 368?
idxs_zero = (mask1d.shape[-1] - 320) // 2 # e.g. zero first/last (368-320)/2=24 indices
mask1d[:idxs_zero], mask1d[-idxs_zero:] = 0, 0

# create 2d mask. zero pad if dimensions don't line up - is this necessary?
mask2d = np.repeat(mask1d[None,:], slice_ksp.shape[1], axis=0)#.astype(int)
mask2d = np.pad(mask2d, ((0,),((slice_ksp.shape[-1]-mask2d.shape[-1])//2,)), mode='constant')

# convert shape e.g. (368,) --> (1, 1, 368, 1)
mask = np_to_tt(np.array([[mask2d[0][np.newaxis].T]])).type(torch.FloatTensor)
print('under-sampling factor:', round(len(mask1d) / sum(mask1d), 2))

under-sampling factor: 8.98


### Set up ConvDecoder

In [70]:
arch_name = 'ConvDecoder'

in_size = [8,4]
out_size = slice_ksp.shape[1:] # shape of (x,y) image slice, e.g. (640, 368)
out_depth = slice_ksp.shape[0]*2 # 2*n_c, i.e. 2*15=30 if multi-coil
num_layers = 8
strides = [1]*(num_layers-1)
num_channels = 160
kernel_size = 3

net = convdecoder(in_size, out_size, out_depth, num_layers, strides, num_channels).type(dtype)

print('# parameters of {}:'.format(arch_name),num_params(net))
#print(net)

[(15, 8), (28, 15), (53, 28), (98, 53), (183, 102), (343, 193), (640, 368)]
# parameters of ConvDecoder: 1850560


### Fit ConvDecoder

##### TODO's
- clean up unnecessary conversions b/w numpy, torch tensor [C,H,W], and torch var [1,C,H,W]
- make separate function that returns net_input given the appropriate scale_factor, i.e. split up mri_helpers.get_scale_factor() into two different functions

In [71]:
# fix the scaling b/w original image and random output image = net(input tensor w values ~U[0,1]) 
# e.g. scale_factor = 168813
# note: can be done using the under-sampled kspace, but we use the full kspace
scale_factor, net_input = get_scale_factor(net,
                                   num_channels,
                                   in_size,
                                   slice_ksp)
slice_ksp = slice_ksp * scale_factor # original fit_untrained() f'n returns this
slice_ksp_tt = np_to_tt(slice_ksp)
    
# mask the kspace
ksp_masked_tt = apply_mask(slice_ksp_tt, mask=mask)
# convert to torch variable [C, W, H] --> [1, C, W, H]
ksp_masked_tt = np_to_var(ksp_masked_tt.data.cpu().numpy()).type(dtype)

# perform ifft of masked kspace
img_masked = ifft_2d(ksp_masked_tt[0]).cpu().numpy()
# reshape complex channels to be adjacent: (15,x,y,2) --> (1,30,x,y)
out = []
for img in img_masked:
    out += [img[:,:,0], img[:,:,1]]
img_masked = np_to_var(np.array(out)).type(dtype)

### Fit network via `fit(...)`

##### Returns
- net: the best network. network output is in image space but not computed
- mse_wrt_ksp = mse(ksp_masked, fft(out) * mask)
- mse_wrt_img = mse(img_masked, out)

##### args:
- `ksp_masked`: ksp_masked_tt, i.e. masked k-space of single slice
- `img_masked`: ifft(ksp_masked)
- `img_ls`: least-squares recon of original (unmasked) k-space. This is used only to compute ssim, psnr, and norm_ratio across number of iterations. Too see how this is created, refer to original ipynb for definining `lsimg`
- `find_best`: whether or not to save best network at each iteration. If set to False, I think `fit()` would just return as output the original net? Not sure why you'd ever want to do that...

Note: Original code has opt_input argument (default False) which would hence return a new version of net_input

##### TODO's (in fit.py)
- make apply_f call less confusing. compare forwardm to utils.transform.apply_mask()
- understand why we backprop on loss_ksp and not loss_img

In [72]:
_, _, _, mse_wrt_ksp, mse_wrt_img, _, net = fit(
        ksp_masked=ksp_masked_tt, img_masked=img_masked,
        net=net, net_input=net_input, mask=mask2d,
        img_ls=None, num_iter=100)

optimize with adam 0.01


# TODO: start here

### Perform data consistency step